<a href="https://colab.research.google.com/github/cmlins/fast_track/blob/main/Engenharia%20de%20dados/desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio 1

## Votação Eleições 2014

### Imports de bibliotecas

In [ ]:
# Instalar o plotly

!pip install --upgrade plotly

     |████████████████████████████████| 13.2MB 274kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
# Instalar o pyspark

!pip install pyspark

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 16.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=a2d192294d9055a2dca276b03ae03e26f3a0d52ac7f14153b06271e7179bb9b6
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
# import das bibliotecas usadas

import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, isnull, asc, desc, mean

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Cria uma sessão spark
spark = SparkSession.builder.master('local').appName('DataWrangling').getOrCreate()

# Configura para que a visualização das saídas seja similar ao Pandas
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [ ]:
#montar o drive
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


### Lista de todos os candidatos a presidente nas eleições de 2014

In [ ]:
candidatos = spark.read.csv('/drive/MyDrive/Desafio/receitas_candidatos_2014_brasil_english.csv', header=True)
# candidatos.limit(5)

In [ ]:
candidatos = candidatos.select('cat_candidate_name', 'id_candidate_num', 'cat_party', 'cat_political_office')
presidentes = candidatos.filter(candidatos.cat_political_office == 'Presidente')
presidentes = presidentes.dropDuplicates()
presidentes.show()

+--------------------+----------------+---------+--------------------+
|  cat_candidate_name|id_candidate_num|cat_party|cat_political_office|
+--------------------+----------------+---------+--------------------+
|JOSÉ LEVY FIDELIX...|              28|     PRTB|          Presidente|
| LUCIANA KREBS GENRO|              50|     PSOL|          Presidente|
|   RUI COSTA PIMENTA|              29|      PCO|          Presidente|
|JOSÉ MARIA DE ALM...|              16|     PSTU|          Presidente|
|   JOSE MARIA EYMAEL|              27|     PSDC|          Presidente|
|EDUARDO JORGE MAR...|              43|       PV|          Presidente|
|MARIA OSMARINA MA...|              40|      PSB|          Presidente|
|     MAURO LUÍS IASI|              21|      PCB|          Presidente|
|EDUARDO HENRIQUE ...|              40|      PSB|          Presidente|
|AÉCIO NEVES DA CUNHA|              45|     PSDB|          Presidente|
|EVERALDO DIAS PER...|              20|      PSC|          Presidente|
| DILM

In [ ]:
candidatos.printSchema()

root
 |-- cat_candidate_name: string (nullable = true)
 |-- id_candidate_num: string (nullable = true)
 |-- cat_party: string (nullable = true)
 |-- cat_political_office: string (nullable = true)



### Votos que cada candidato à Presidência fez na região Sul do Brasil

In [ ]:
#  Carrega tabela com os votos das eleições

votacao = spark.read.csv('/drive/MyDrive/Desafio/votacao_brasil_english.csv',header=True)
votacao.limit(5)

id_candidate_seq,id_candidate_num,cat_candidate_name,cat_state,cat_party,cat_coalition,cat_composition,cat_eletion_result,num_turn,num_votes
10000000001,111,GLADSON DE LIMA C...,AC,PP,ALIANÇA POR UM AC...,PMDB / PSDB / PT ...,ELEITO,1,218756
10000000003,45,MÁRCIO MIGUEL BITTAR,AC,PSDB,ALIANÇA POR UM AC...,PMDB / PSDB / PT ...,2º TURNO,1,116948
10000000003,45,MÁRCIO MIGUEL BITTAR,AC,PSDB,ALIANÇA POR UM AC...,PMDB / PSDB / PT ...,NÃO ELEITO,2,186658
10000000021,12123,HENRY ANTONIO SIL...,AC,PDT,PARTIDO ISOLADO,PDT,SUPLENTE,1,2605
10000000022,12012,ZENILDO VIEIRA DA...,AC,PDT,PARTIDO ISOLADO,PDT,NÃO ELEITO,1,0


In [ ]:
# seleção das colunas necessárias para a avaliação

tb_votacao = votacao[['id_candidate_num', 'cat_candidate_name', 'cat_state', 'cat_party', 'num_turn', 'num_votes']]
tb_votacao

id_candidate_num,cat_candidate_name,cat_state,cat_party,num_turn,num_votes
111,GLADSON DE LIMA C...,AC,PP,1,218756
45,MÁRCIO MIGUEL BITTAR,AC,PSDB,1,116948
45,MÁRCIO MIGUEL BITTAR,AC,PSDB,2,186658
12123,HENRY ANTONIO SIL...,AC,PDT,1,2605
12012,ZENILDO VIEIRA DA...,AC,PDT,1,0
12570,JOSÉ ALVES DA SILVA,AC,PDT,1,47
12456,MARCOS GLAY RIBEI...,AC,PDT,1,172
12111,JOSÉ ALTANIZIO TA...,AC,PDT,1,2196
12147,AURELIANO ALVES R...,AC,PDT,1,486
12580,JOSE AUGUSTO PINH...,AC,PDT,1,1454


In [ ]:
# Seleção dos votos dos estados do Sul

tb_votacao_sul = tb_votacao[(tb_votacao['cat_state'] == 'RS') | (tb_votacao['cat_state'] == 'SC') | (tb_votacao['cat_state'] == 'PR')]
tb_votacao_sul.limit(5)

id_candidate_num,cat_candidate_name,cat_state,cat_party,num_turn,num_votes
50069,XÊNIA KAROLINE MELLO,PR,PSOL,1,2399
50,BERNARDO SEIXAS P...,PR,PSOL,1,35327
5055,ANDERSON ANDREAZZA,PR,PSOL,1,1438
5090,SIRINEU ZANCHI ME...,PR,PSOL,1,568
5050,THIAGO DE SOUSA B...,PR,PSOL,1,5366


In [ ]:
# candidatos.columns

In [ ]:
# Renomeia as colunas da tabela com os cadidatos à presidência

presidentes_renamed = presidentes.withColumnRenamed('cat_candidate_name','cat_candidate_name_pres').withColumnRenamed('id_candidate_num','id_candidate_num_pres').withColumnRenamed('cat_party','cat_party_pres').withColumnRenamed('cat_political_office','cat_political_office_pres')

# 'cat_candidate_name','id_candidate_num','cat_party','cat_political_office'
# 'cat_candidate_name_pres','id_candidate_num_pres','cat_party_pres','cat_political_office_pres'

In [ ]:
# Join das tabelas dos cadidatos à Presidência com a tabela dos votos nos estados do Sul usando como referência os nomes do candidatos

join = tb_votacao_sul.join(presidentes_renamed, tb_votacao_sul.cat_candidate_name == presidentes_renamed.cat_candidate_name_pres)

In [ ]:
#  Seleção dos votos somente do primeiro turno

primeiro_turno = join.filter(join.num_turn == 1)
primeiro_turno.show()

+----------------+--------------------+---------+---------+--------+---------+-----------------------+---------------------+--------------+-------------------------+
|id_candidate_num|  cat_candidate_name|cat_state|cat_party|num_turn|num_votes|cat_candidate_name_pres|id_candidate_num_pres|cat_party_pres|cat_political_office_pres|
+----------------+--------------------+---------+---------+--------+---------+-----------------------+---------------------+--------------+-------------------------+
|              28|JOSÉ LEVY FIDELIX...|       SC|     PRTB|       1|    15774|   JOSÉ LEVY FIDELIX...|                   28|          PRTB|               Presidente|
|              28|JOSÉ LEVY FIDELIX...|       RS|     PRTB|       1|    17954|   JOSÉ LEVY FIDELIX...|                   28|          PRTB|               Presidente|
|              28|JOSÉ LEVY FIDELIX...|       PR|     PRTB|       1|    33612|   JOSÉ LEVY FIDELIX...|                   28|          PRTB|               Presidente|
|   

In [ ]:
# Drop das colunas desnecessárias

pres_sul = primeiro_turno.drop('cat_candidate_name_pres','id_candidate_num_pres','cat_party_pres','cat_political_office_pres')
pres_sul.show()

+----------------+--------------------+---------+---------+--------+---------+
|id_candidate_num|  cat_candidate_name|cat_state|cat_party|num_turn|num_votes|
+----------------+--------------------+---------+---------+--------+---------+
|              28|JOSÉ LEVY FIDELIX...|       SC|     PRTB|       1|    15774|
|              28|JOSÉ LEVY FIDELIX...|       RS|     PRTB|       1|    17954|
|              28|JOSÉ LEVY FIDELIX...|       PR|     PRTB|       1|    33612|
|              50| LUCIANA KREBS GENRO|       SC|     PSOL|       1|    53532|
|              50| LUCIANA KREBS GENRO|       RS|     PSOL|       1|   143370|
|              50| LUCIANA KREBS GENRO|       PR|     PSOL|       1|    79267|
|              29|   RUI COSTA PIMENTA|       SC|      PCO|       1|      390|
|              29|   RUI COSTA PIMENTA|       RS|      PCO|       1|      505|
|              29|   RUI COSTA PIMENTA|       PR|      PCO|       1|      560|
|              16|JOSÉ MARIA DE ALM...|       SC|   

In [ ]:
# Votos dos candidatos à presidência em Santa Catarina no primeiro turno em 2014

sc = pres_sul.filter(pres_sul.cat_state == 'SC')
sc.show()

+----------------+--------------------+---------+---------+--------+---------+
|id_candidate_num|  cat_candidate_name|cat_state|cat_party|num_turn|num_votes|
+----------------+--------------------+---------+---------+--------+---------+
|              28|JOSÉ LEVY FIDELIX...|       SC|     PRTB|       1|    15774|
|              50| LUCIANA KREBS GENRO|       SC|     PSOL|       1|    53532|
|              29|   RUI COSTA PIMENTA|       SC|      PCO|       1|      390|
|              16|JOSÉ MARIA DE ALM...|       SC|     PSTU|       1|     3975|
|              27|   JOSE MARIA EYMAEL|       SC|     PSDC|       1|     2005|
|              43|EDUARDO JORGE MAR...|       SC|       PV|       1|    30239|
|              40|MARIA OSMARINA MA...|       SC|      PSB|       1|   476430|
|              21|     MAURO LUÍS IASI|       SC|      PCB|       1|     1413|
|              45|AÉCIO NEVES DA CUNHA|       SC|     PSDB|       1|  1963772|
|              20|EVERALDO DIAS PER...|       SC|   

In [ ]:
# Votos dos candidatos à presidência no Rio Grande do Sul no primeiro turno em 2014

rs = pres_sul.filter(pres_sul.cat_state == 'RS')
rs.show()

+----------------+--------------------+---------+---------+--------+---------+
|id_candidate_num|  cat_candidate_name|cat_state|cat_party|num_turn|num_votes|
+----------------+--------------------+---------+---------+--------+---------+
|              28|JOSÉ LEVY FIDELIX...|       RS|     PRTB|       1|    17954|
|              50| LUCIANA KREBS GENRO|       RS|     PSOL|       1|   143370|
|              29|   RUI COSTA PIMENTA|       RS|      PCO|       1|      505|
|              16|JOSÉ MARIA DE ALM...|       RS|     PSTU|       1|     4083|
|              27|   JOSE MARIA EYMAEL|       RS|     PSDC|       1|     2255|
|              43|EDUARDO JORGE MAR...|       RS|       PV|       1|    43703|
|              40|MARIA OSMARINA MA...|       RS|      PSB|       1|   732828|
|              21|     MAURO LUÍS IASI|       RS|      PCB|       1|     2301|
|              45|AÉCIO NEVES DA CUNHA|       RS|     PSDB|       1|  2639647|
|              20|EVERALDO DIAS PER...|       RS|   

In [ ]:
# # Votos dos candidatos à presidência no Paraná no primeiro turno em 2014

pr = pres_sul.filter(pres_sul.cat_state == 'PR')
pr.show()

+----------------+--------------------+---------+---------+--------+---------+
|id_candidate_num|  cat_candidate_name|cat_state|cat_party|num_turn|num_votes|
+----------------+--------------------+---------+---------+--------+---------+
|              28|JOSÉ LEVY FIDELIX...|       PR|     PRTB|       1|    33612|
|              50| LUCIANA KREBS GENRO|       PR|     PSOL|       1|    79267|
|              29|   RUI COSTA PIMENTA|       PR|      PCO|       1|      560|
|              16|JOSÉ MARIA DE ALM...|       PR|     PSTU|       1|     4552|
|              27|   JOSE MARIA EYMAEL|       PR|     PSDC|       1|     4828|
|              43|EDUARDO JORGE MAR...|       PR|       PV|       1|    44755|
|              40|MARIA OSMARINA MA...|       PR|      PSB|       1|   861820|
|              21|     MAURO LUÍS IASI|       PR|      PCB|       1|     2409|
|              45|AÉCIO NEVES DA CUNHA|       PR|     PSDB|       1|  3021988|
|              20|EVERALDO DIAS PER...|       PR|   

## Gráficos dos votos à Presidência, em 2014, no 1º turno nos estados do Sul do Brasil

In [ ]:
fig = px.pie(rs.toPandas(), values='num_votes', names='cat_candidate_name', title='RS')
fig.show()

In [ ]:
fig = px.pie(sc.toPandas(), values='num_votes', names='cat_candidate_name', title='SC')
fig.show()

In [ ]:
fig = px.pie(pr.toPandas(), values='num_votes', names='cat_candidate_name', title='PR')
fig.show()

In [ ]:
labels = rs.select('cat_candidate_name').toPandas()['cat_candidate_name'].tolist()
num_votes_rs = rs.select('num_votes').toPandas()['num_votes'].tolist()
num_votes_sc = sc.select('num_votes').toPandas()['num_votes'].tolist()
num_votes_pr = pr.select('num_votes').toPandas()['num_votes'].tolist()

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]], subplot_titles=['RS', 'SC', 'PR'])
fig.add_trace(go.Pie(labels=labels, values=num_votes_rs, name="RS"), 
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=num_votes_sc, name="SC"),
              1, 2)
fig.add_trace(go.Pie(labels=labels, values=num_votes_pr, name="PR"),
              1, 3)

fig.update_layout(title_text='Votações no primeiro turno (SUL)')

### Algumas referências

1. https://plotly.com/python/pie-charts/  
2. https://analyticsindiamag.com/how-to-use-plotly-in-colab/  
3. https://plotly.com/javascript/pie-charts/#:~:text=In%20order%20to%20create%20pie%20chart%20subplots%2C%20you,a%20rectangle%20defined%20by%20X%20and%20Y%20arrays  
4. https://www.kaggle.com/srivignesh/an-introduction-to-pyspark-apache-spark-in-python
5. https://www.cetax.com.br/blog/tutorial-pyspark-e-mllib/
